# import

In [1]:
import torchvision
import torch

import os
import numpy as np
from PIL import Image
from torch.utils import data
from torch.utils.data import DataLoader #데이터 불러오기
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
#훈련
from torch import nn
from torch.nn import functional as F
#시각화
import matplotlib.pyplot as plt


In [2]:
print(torch.__version__)

1.8.1


In [3]:
import os

import torch
import numpy as np
import matplotlib.pyplot as plt


def listFiles(rootdir='.', suffix='png'):
    """Performs recursive glob with given suffix and rootdir
        :param rootdir is the root directory
        :param suffix is the suffix to be searched as PNG or JPG
    """
    return [os.path.join(looproot, filename)
        for looproot, _, filenames in os.walk(rootdir)
        for filename in filenames if filename.endswith(suffix)]

cityscapes_valid_classes = [7, 8, 11, 12, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33]
class_map = dict(zip( range(19),cityscapes_valid_classes))

def convertTrainIdToClassId(img):
         temp=np.copy(img)
         for trainID in range(19):
             #print(trainID," " ,class_map[trainID])
             temp[img==trainID]=class_map[trainID]
             
         return temp


def get_cityscapes_labels():
    return np.array([
         #[  0,   0,   0],
        [128, 64, 128],
        [244, 35, 232],
        [70, 70, 70],
        [102, 102, 156],
        [190, 153, 153],
        [153, 153, 153],
        [250, 170, 30],
        [220, 220, 0],
        [107, 142, 35],
        [152, 251, 152],
        [0, 130, 180],
        [220, 20, 60],
        [255, 0, 0],
        [0, 0, 142],
        [0, 0, 70],
        [0, 60, 100],
        [0, 80, 100],
        [0, 0, 230],
        [119, 11, 32]
        ])



def encode_segmap(mask):
    """Encode segmentation label images as pascal classes
    Args:
        mask (np.ndarray): raw segmentation label image of dimension
          (M, N, 3), in which the Pascal classes are encoded as colours.
    Returns:
        (np.ndarray): class map with dimensions (M,N), where the value at
        a given location is the integer denoting the class index.
    """
    mask = mask.astype(int)
    label_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.int16)
    for ii, label in enumerate(get_cityscapes_labels()):#get_pascal_labels()
        label_mask[np.where(np.all(mask == label, axis=-1))[:2]] = ii
    label_mask = label_mask.astype(int)
    return label_mask


def decode_seg_map_sequence(label_masks, dataset='pascal'):
    rgb_masks = []
    for label_mask in label_masks:
        rgb_mask = decode_segmap(label_mask, dataset)
        rgb_masks.append(rgb_mask)
    rgb_masks = torch.from_numpy(np.array(rgb_masks).transpose([0, 3, 1, 2]))
    return rgb_masks

def decode_segmap(label_mask, dataset, plot=False):
    """Decode segmentation class labels into a color image
    Args:
        label_mask (np.ndarray): an (M,N) array of integer values denoting
          the class label at each spatial location.
        plot (bool, optional): whether to show the resulting color image
          in a figure.
    Returns:
        (np.ndarray, optional): the resulting decoded color image.
    """
    if dataset == 'pascal':
      print()
    elif dataset == 'cityscapes':
        n_classes = 19
        label_colours = get_cityscapes_labels()
    else:
        raise NotImplementedError

    r = label_mask.copy()
    g = label_mask.copy()
    b = label_mask.copy()
    for ll in range(0, n_classes):
        r[label_mask == ll] = label_colours[ll, 0]
        g[label_mask == ll] = label_colours[ll, 1]
        b[label_mask == ll] = label_colours[ll, 2]
    
    r[label_mask == 255] = 0
    g[label_mask == 255] = 0
    b[label_mask == 255] =0
    
    rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
   # rgb = np.zeros((label_mask.shape[1], label_mask.shape[2], 3))
    #replace blue with red as opencv uses bgr
    rgb[:, :, 0] = r /255.0     
    rgb[:, :, 1] = g /255.0
    rgb[:, :, 2] = b /255.0
#    
#    rgb = np.zeros((label_mask.shape[1], label_mask.shape[2], 3))
#    #replace blue with red as opencv uses bgr
#    rgb[:, :, 0] = b #/255.0     
#    rgb[:, :, 1] = g #/255.0
#    rgb[:, :, 2] = r #/255.0
#    
    if plot:
        plt.imshow(rgb)
        plt.show()
    else:
        return rgb
def decode_segmap_cv(label_mask, dataset, plot=False):
    """Decode segmentation class labels into a color image
    Args:
        label_mask (np.ndarray): an (M,N) array of integer values denoting
          the class label at each spatial location.
        plot (bool, optional): whether to show the resulting color image
          in a figure.
    Returns:
        (np.ndarray, optional): the resulting decoded color image.
    """
    if dataset == 'pascal':
      print()
    elif dataset == 'cityscapes':
        n_classes = 19
        label_colours = get_cityscapes_labels()
    else:
        raise NotImplementedError

    r = label_mask.copy()
    g = label_mask.copy()
    b = label_mask.copy()
    for ll in range(0, n_classes):
        r[label_mask == ll] = label_colours[ll, 0]
        g[label_mask == ll] = label_colours[ll, 1]
        b[label_mask == ll] = label_colours[ll, 2]
    
    r[label_mask == 255] = 0
    g[label_mask == 255] = 0
    b[label_mask == 255] =0
    
#    rgb = np.zeros((label_mask.shape[0], label_mask.shape[1], 3))
#   # rgb = np.zeros((label_mask.shape[1], label_mask.shape[2], 3))
#    #replace blue with red as opencv uses bgr
#    rgb[:, :, 0] = r /255.0     
#    rgb[:, :, 1] = g /255.0
#    rgb[:, :, 2] = b /255.0
#    
    rgb = np.zeros((label_mask.shape[1], label_mask.shape[2], 3))
    #replace blue with red as opencv uses bgr
    rgb[:, :, 0] = b #/255.0     
    rgb[:, :, 1] = g #/255.0
    rgb[:, :, 2] = r #/255.0
#    
    if plot:
        plt.imshow(rgb)
        plt.show()
    else:
        return rgb
def generate_param_report(logfile, param):
    log_file = open(logfile, 'w')
    for key, val in param.items():
        log_file.write(key + ':' + str(val) + '\n')
    log_file.close()


def lr_poly(base_lr, iter_, max_iter=100, power=0.9):
    return base_lr * ((1 - float(iter_) / max_iter) ** power)


    
from torchvision import transforms 

if __name__ == '__main__':
    print()
    ar=np.array([[0,7,10],[7,3,6]])
    z=convertTrainIdToClassId(ar)
#    img3= transforms.ToPILImage()(torch.from_numpy(ou).type(torch.FloatTensor))#.detach().cpu()
#    img3.save(oupath)
    print(z)


[[ 7 20 23]
 [20 12 19]]


# data loader

## Cityscapes code

In [6]:

#from torch.utils.data import TensorDataset, DataLoader
#from dataloader.utils import listFiles #모르겠음

class Cityscapes(data.Dataset):

    def __init__(self, root='path', split="train", transform=None,extra=False):
        """
        Cityscapes dataset folder has two folders, 'leftImg8bit' folder for images and 'gtFine_trainvaltest' 
        folder for annotated images with fine annotations 'labels'.
        """
        self.root = os.getenv('HOME')+'/aiffel/0siaiffel_project/path/lite_data/'
        self.split = split #train, validation, and test sets
        self.transform = transform
        self.files = {}
        self.n_classes = 19
        self.extra=extra

        if not self.extra:
            print("Using fine dataset")
            self.images_path = os.path.join(self.root, 'leftImg8bit(real)', self.split)
            self.labels_path = os.path.join(self.root, 'gtFine(seg)', self.split)
        else:
            print("Using Coarse dataset")

            self.images_path = os.path.join(self.root, 'leftImg8bit', self.split)
            self.labels_path = os.path.join(self.root, 'gtCoarse', 'gtCoarse', self.split)            
            
        #print(self.images_path)
        self.files[split] = listFiles(rootdir=self.images_path, suffix='.png')#list of the pathes to images

        self.void_classes = [0, 1, 2, 3, 4, 5, 6, 9, 10, 14, 15, 16, 18, 29, 30, -1] #not to train
        self.valid_classes = [7, 8, 11, 12, 13, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33]
        self.class_names = ['road', 'sidewalk', 'building', 'wall', 'fence', \
                            'pole', 'traffic_light', 'traffic_sign', 'vegetation', 'terrain', \
                            'sky', 'person', 'rider', 'car', 'truck', 'bus', 'train', \
                            'motorcycle', 'bicycle']
        
        self.ignore_index = 255
        self.class_map = dict(zip(self.valid_classes, range(self.n_classes)))
        #print(self.class_map)
        
        if not self.files[split]:
            raise Exception("No files for split=[%s] found in %s" % (split, self.images.path))

        print("Found %d %s images" % (len(self.files[split]), split))
        
    
    def __len__(self):
        return len(self.files[self.split])
    
    def __getitem__(self, index):
        image_path = self.files[self.split][index].rstrip()
        #print(image_path)
        if not self.extra:
            label_path = os.path.join(self.labels_path,
                                image_path.split(os.sep)[-2],
                                os.path.basename(image_path)[:-15] + 'gtFine_labelIds.png')
        else:
            label_path = os.path.join(self.labels_path,
                                image_path.split(os.sep)[-2],
                                os.path.basename(image_path)[:-15] + 'gtCoarse_labelIds.png')
        _img = Image.open(image_path).convert('RGB')
        _tmp = np.array(Image.open(label_path), dtype=np.uint8)
        _tmp = self.encode_segmap(_tmp)

        _target = Image.fromarray(_tmp)

        sample = {'image': _img, 'label': _target}

        if self.transform:
            sample = self.transform(sample)
        return sample
    
    def encode_segmap(self, mask):
        # Put all void classes to ignore_index
        for _voidc in self.void_classes:
            mask[mask == _voidc] = self.ignore_index
        for _validc in self.valid_classes:
            mask[mask == _validc] = self.class_map[_validc]
        return mask

print("ok")

ok


In [7]:
abcd=Cityscapes()

Using fine dataset
Found 2975 train images


# encoder-decoder

In [ ]:

import torch.nn as nn
import torch.nn.functional as F
import torch
import math

from models.backbone_networks import MobileNetV2
from models import aspp # daspp용으로 사용
from models.separableconv import SeparableConv2d #depwise seperable conv




class RT(nn.Module):
    
    def __init__(self, n_classes=19,PRETRAINED_WEIGHTS=".", pretrained=True):
        
        super(RT, self).__init__()
        print("LiteSeg-MobileNet...")

        self.mobile_features=MobileNetV2.MobileNetV2() # 백본 모델 
        if pretrained:
            state_dict = torch.load(PRETRAINED_WEIGHTS)
            self.mobile_features.load_state_dict(state_dict)
        
        rates = [1, 3, 6, 9] #aspp rate 


        self.aspp1 = aspp.ASPP(1280, 96, rate=rates[0]) #1280 (in_channels) 96(out_channels)
        self.aspp2 = aspp.ASPP(1280, 96, rate=rates[1])
        self.aspp3 = aspp.ASPP(1280, 96, rate=rates[2])
        self.aspp4 = aspp.ASPP(1280, 96, rate=rates[3])

        self.relu = nn.ReLU()
        self.global_avg_pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)), #fc 연산 대체 
                                             nn.Conv2d(1280, 96, 1, stride=1, bias=False),
                                             nn.BatchNorm2d(96),
                                             nn.ReLU())
        #self.conv1 = nn.Conv2d(480+1280, 96, 1, bias=False)
        self.conv1 =SeparableConv2d(480+1280,96,1) # input채널에 480 더하고 separable conv실행 
        self.bn1 = nn.BatchNorm2d(96) 

        #adopt [1x1, 48] for channel reduction.----??  이게 뭐지
        #self.conv2 = nn.Conv2d(24, 32, 1, bias=False) #여기서 들어온 24랑 output채널 96을 더한다. 왜?
        #self.bn2 = nn.BatchNorm2d(32)
    
     # 이것도 3*3aspp하고 난후 3*3 같음 (근데 3*3conv가 하나 없다. 24+96에 두개가 있는건가?)
        self.last_conv = nn.Sequential(#nn.Conv2d(24+96, 96, kernel_size=3, stride=1, padding=1, bias=False),
                                       SeparableConv2d(24+96,96,3,1,1), #3*3 conv
                                       nn.BatchNorm2d(96),
                                       nn.ReLU(),
                                       #nn.Conv2d(96, 96, kernel_size=3, stride=1, padding=1, bias=False),
                                       SeparableConv2d(96,96,3,1,1), #3*3conv
                                       nn.BatchNorm2d(96),
                                       nn.ReLU(),    #n classes -19
                                       nn.Conv2d(96, n_classes, kernel_size=1, stride=1)) #1*1conv (average뒤에)
        
    def forward(self, input):
        x, low_level_features = self.mobile_features(input)
        #print(x.size())
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.interpolate(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)                                   #x.size()[1:] ->배치dim제외하고 all dim이란 의미 2는 뭐지?
        x = torch.cat((x,x1, x2, x3, x4, x5), dim=1) #이건 aspp의 output+high dim = concat 1
        #print('after aspp cat',x.size())
        x = self.conv1(x) #1*1 conv
        x = self.bn1(x) 
        x = self.relu(x)
        #upsampling*8(아래 계산이 8배 하는 수식인가?)
        x = F.interpolate(x, size=(int(math.ceil(input.size()[-2]/4)),
                                int(math.ceil(input.size()[-1]/4))), mode='bilinear', align_corners=True) #upsampling*8
       
    # ablation=torch.max(low_level_features, 1)[1]
        #print('after con on aspp output',x.size())

        ##comment to remove low feature
        #low_level_features = self.conv2(low_level_features)
        #low_level_features = self.bn2(low_level_features)
        #low_level_features = self.relu(low_level_features)
        #print("low",low_level_features.size())
        
        x = torch.cat((x, low_level_features), dim=1)  #upsampling한 x값 +백본 low에서 뽑은 feature= concat
        #print('after cat low feature with output of aspp',x.size())

        x = self.last_conv(x) #decoder(3*3 2개+1*1 1개)
        x = F.interpolate(x, size=input.size()[2:], mode='bilinear', align_corners=True) #upsamling*4

        return x#,ablation

    def freeze_bn(self): #앤뭐지?
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    def __init_weight(self): #이건뭐지?
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()